In [1]:
import os
from simplified_scrapy import SimplifiedDoc, utils

path = 'data/one_run/m'
#read the whole text from 
myList = []
for root, dirs, files in os.walk(path):
    
    for file in files:
        if file.endswith('osm.net.xml'):
            myList.append(os.path.join(root, file))

for file in myList:
    xml = utils.getFileContent(file)
    doc = SimplifiedDoc(xml)
    arg = [ (e['tl'], e['linkIndex'],e['dir'],e['via']) for e in doc.selects('connection')]

print(arg)

    
arg = list(dict.fromkeys(arg))

for file in myList:
    xml = utils.getFileContent(file)
    doc = SimplifiedDoc(xml)
    tl = [ e['id'] for e in doc.selects('tlLogic')]


import pandas as pd
data = pd.DataFrame(columns= ["tl","index","dir","via"], index=range(len(arg)))
l = []
for i in range(len(arg)):
    l.append(arg[i][0] )
    if arg[i][0] == tl[0]:
        data.iloc[i,0] = arg[i][0]
        data.iloc[i,3] = arg[i][3]
        data.iloc[i,1] = arg[i][1]
        data.iloc[i,2] = arg[i][2]
        
data.dropna(inplace=True)
data.iloc[:,3] = data.iloc[:,3].str.replace(tl[0],'')
data = data.sort_values(by='index')
data.reset_index(inplace=True,drop=True)
data = data.drop(columns=['tl'])
direction_lane_dict = {}
for i in data.groupby('dir'):
    d = i[0].upper()+'G'
    direction_lane_dict[d] = [int(x) for x in i[1]['index']]
    
for file in myList:
    xml = utils.getFileContent(file)
    doc = SimplifiedDoc(xml)
    arg = [ (e['tl'],e['from'], e['to'],e['fromLane'],e['toLane']) for e in doc.selects('connection')]
            
arg = list(dict.fromkeys(arg))

import pandas as pd
data = pd.DataFrame(columns= ["from","to","froml","tol"], index=range(len(arg)))
for i in range(len(arg)):
    if arg[i][0] == tl[0]:
        data.iloc[i,0] = arg[i][1]
        data.iloc[i,1] = arg[i][2]
        data.iloc[i,2] = arg[i][3]
        data.iloc[i,3] = arg[i][4]
        
data.dropna(inplace=True)
data['from'] = data['from']+'_'+data['froml']
data['to'] = data['to']+'_'+data['tol'] 
l = list(data['from'].unique())
for i in data['to'].unique():
    l.append(i)

node_light_7 = tl[0]
listLanes = set(l)
#rint(listLanes)

direction_list = [k for k in direction_lane_dict]

for file in myList:
    xml = utils.getFileContent(file)
    doc = SimplifiedDoc(xml)
    arg = [ e['state'] for e in doc.selects('phase')]
         
arg = list(dict.fromkeys(arg))
green = []
lenpahse = len(arg[0])
for i in arg: 
    if 'g' in i or 'G' in i:
        green.append(i)
        
phases_names =[]
phases = {}
controlSignal = []

for i in green:
    p = ''
    for j in range(len(i)):
        if i[j]=='G' or i[j]=='g':
            for key in direction_lane_dict:
                if j+1 in direction_lane_dict[key]:
                    if p == '':
                        p = p + key
                    else:
                        p = p + '_' +key

    if p not in phases_names:
        phases[p] = i
        controlSignal.append(i)
        phases_names.append(p)

phases_light_7 = [p for p in phases]
actions= list(phases.values())
numphases = len(phases_light_7)
Durations=[7,3,7,7,7,7,7,3,7]

[(None, None, 'r', ':cluster_1269313230_9243405806_0_0'), (None, None, 's', ':cluster_1269313230_9243405806_1_0'), (None, None, 'l', ':cluster_1269313230_9243405806_2_0'), (None, None, 't', ':cluster_1269313230_9243405806_3_0'), (None, None, 't', ':7695994307_0_0'), (None, None, 'r', ':9243624879_0_0'), (None, None, 'l', ':9243624879_1_0'), (None, None, 't', ':9243624879_2_0'), (None, None, 's', ':1262555922_2_0'), (None, None, 's', ':2054450562_0_0'), (None, None, 'l', ':2054450562_1_0'), (None, None, 's', ':1262556010_0_0'), (None, None, 'l', ':1262556010_1_0'), (None, None, 't', ':1262556010_2_0'), (None, None, 'r', ':3129788157_3_0'), (None, None, 'l', ':3129788157_4_0'), (None, None, 't', ':3129788157_5_0'), (None, None, 's', ':3129788158_3_0'), (None, None, 'l', ':3129788158_4_0'), (None, None, 't', ':3129788158_5_0'), (None, None, 'r', ':1250291427_0_0'), (None, None, 'l', ':1250291427_1_0'), (None, None, 't', ':1250291427_2_0'), (None, None, 's', ':1250291421_0_0'), (None, None

In [2]:
import os, sys
import statistics 
#if 'SUMO_HOME' in os.environ:
#    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
#    sys.path.append(tools)
#else:
#    sys.exit("please declare environment variable 'SUMO_HOME'")

import traci
import sumolib
from tqdm import tqdm
import random
import time

import CH
import PredictPos
import ModifyFile

f_memory = open("memorybasics.csv", "w")
f_memory.write('Information,x,y' + "\n")
f_memory.close()
sumoBinary = "/Users/remi/sumo/bin/sumo-gui"
sumoCmd = [sumoBinary,'-c','data/one_run/m/osm.sumocfg','-n', 'data/one_run/m/osm.net.xml']

time_start = time.time()
traci.start(sumoCmd)

########## Position of Traffic Light ##########

net = sumolib.net.readNet('data/one_run/m/osm.net.xml')

node_ids = [node.getID() for node in net.getNodes()]
traffic_pos = []

for node in node_ids:
    if (net.getNode(node).getType()) == 'traffic_light':
        traffic_pos.append(net.getNode(node).getCoord())

########## Selection of the lanes and edges ##########

TotalLane = traci.lane.getIDList()
TotalListLane = list(TotalLane)
ListRemove = []

TotalEdge = traci.edge.getIDList()
TotalBigEdge = list(TotalEdge)

for edge in TotalEdge:
    if(traci.edge.getLaneNumber(edge) < 2):
        TotalBigEdge.remove(edge)

allCar = []

for edge in TotalBigEdge:
    for lane in TotalLane:
        if(lane[0:10] in edge):
            if lane in TotalListLane:
                TotalListLane.remove(lane)

TotalLane = tuple(TotalListLane)

TotalLane = tuple(TotalListLane)
for lane in TotalLane:
    if(traci.lane.getMaxSpeed(lane)< 8.3):   #8.34 = 30 km/h
        TotalListLane.remove(lane)

########## Some informations ##########

print('number of TL: ', len(traffic_pos))
print('edge avant: ', len(TotalEdge), 'edge après: ', len(TotalBigEdge))
print('lane avant: ', len(TotalLane), 'lane après: ', len(TotalListLane))

########## Launch of the simulation ##########

count = 2000 #960240
i = 0
while traci.simulation.getTime() <= count:
    
    print(i)
########## Simulation Step ##########

    traci.simulationStep()
    current_time = traci.simulation.getTime()

    action= i%9

########## Traffic Lighter ##########

    duration= Durations[action]    
    traci.trafficlight.setRedYellowGreenState(node_light_7, actions[action])

    Yellow = ''
    for j in range(len(phases_light_7[0])):
        Yellow = Yellow+"y"
        
    for j in range(3):
        timestamp = traci.simulation.getTime()
        traci.trafficlight.setRedYellowGreenState(node_light_7, Yellow)
 
########## List Vehicle ##########

    listVehID = []

    for edge in TotalBigEdge:
        listVeh = traci.edge.getLastStepVehicleIDs(edge)
        listVehID = listVehID + list(listVeh)
    for lane in TotalListLane:
        listVeh = traci.lane.getLastStepVehicleIDs(lane)
        listVehID = listVehID + list(listVeh)

########## CH ##########

    time0 = time.time()

    listNeighbor = []
    listNeighbor = CH.detectNeighbourg(listVehID)
    [listRole, listIDassigned]= CH.roleCar(listVehID,listNeighbor)
    qty = CH.assignColor(listVehID, listRole)

    time1 = time.time()

########## Start Calcul + Memory ##########

    memory_time = '\ntime: %d \n' %traci.simulation.getTime()
    f_memory = open("memorybasics.csv", "a")
    f_memory.write(memory_time + "\n")

    print(listVeh)

    for vehicle in listVehID:
        if vehicle in allCar:
            pass
        else:
            allCar.append(vehicle)
        position = traci.vehicle.getPosition(vehicle)
        futurPos = PredictPos.FuturPos(vehicle)
        memory_vehi = 'vehicle: %s' %vehicle
        memory_pos = 'Current Position:, %f,%f' %(position[0],position[1])
        memory_fut = 'Predict Position:, %f,%f' %(futurPos[0],futurPos[1])
        f_memory.write(memory_vehi + "\n" + memory_pos + "\n" + memory_fut + "\n")

    f_memory.close()

    i = i + 1 

print(allCar)

########## Analyse ##########

for vehicle in allCar:
    ModifyFile.calculationError(vehicle)

ModifyFile.calculationTot()

time_end = time.time()
print ("traci time elapsed: {}".format(time_end-time_start))
traci.close(True)

 Retrying in 1 seconds
number of TL:  6
edge avant:  3312 edge après:  82
lane avant:  2729 lane après:  365
0
()
((1341.41, 1000.46), 0, 'TL')
1
()
((1341.41, 1000.46), 0, 'TL')
2
()
((1341.41, 1000.46), 0, 'TL')
((1341.41, 1000.46), (1108.13, 750.04), 'north')
3
('veh1',)
((1341.41, 1000.46), 0, 'TL')
((1341.41, 1000.46), (1108.13, 750.04), 'north')
4
('veh1',)
((1341.41, 1000.46), 0, 'TL')
((815.88, 1088.35), (697.43, 1187.61), 'west')
((1341.41, 1000.46), (1108.13, 750.04), 'north')
5
('veh1',)
((815.88, 1088.35), (697.43, 1187.61), 'west')
((1341.41, 1000.46), (1108.13, 750.04), 'north')
6
('veh1',)
((815.88, 1088.35), (697.43, 1187.61), 'west')
((1341.41, 1000.46), (1108.13, 750.04), 'north')
7
()
((1341.41, 1000.46), (1108.13, 750.04), 'north')
((1341.41, 1000.46), 0, 'TL')
((815.88, 1088.35), (697.43, 1187.61), 'west')
8
()
((1341.41, 1000.46), (1108.13, 750.04), 'north')
((815.88, 1088.35), (697.43, 1187.61), 'west')
((1341.41, 1000.46), (1108.13, 750.04), 'north')
((1341.41, 